In [10]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.grid_search import GridSearchCV#PARAMETER TUNING


from scipy.sparse import hstack
import string
from sklearn.feature_extraction.text import CountVectorizer#NLP
from scipy.sparse import csr_matrix


G:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
G:\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [11]:
translator = str.maketrans('', '', string.punctuation)
data = pd.read_csv('newdataset.csv',encoding="iso-8859-1")

G:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
data.head()

,ItemID,Sentiment,SentimentSource,SentimentText,Unnamed: 4
0,1.0,0.0,Sentiment140,is so sad for my APL frie...,NaN
1,2.0,0.0,Sentiment140,I missed the New Moon trail...,NaN
2,3.0,1.0,Sentiment140,omg its already 7:30 :O,NaN
3,4.0,0.0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...,NaN
4,5.0,0.0,Sentiment140,i think mi bf is cheating on me!!! ...,NaN


In [13]:
data.drop(['ItemID','SentimentSource','Unnamed: 4'],axis=1,inplace=True)

In [14]:
data.head()

,Sentiment,SentimentText
0,0.0,is so sad for my APL frie...
1,0.0,I missed the New Moon trail...
2,1.0,omg its already 7:30 :O
3,0.0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0.0,i think mi bf is cheating on me!!! ...


In [15]:
data['Sentiment'].value_counts()

1.0    26916
0.0    23073
Name: Sentiment, dtype: int64

# MISSING VALUE TREATMENT

In [16]:
data.dropna(axis=0,inplace=True)

In [17]:
data.isnull().sum()

Sentiment        0
SentimentText    0
dtype: int64

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
#COUNT VECTORIZER for extracting maximum features from SentimentText

In [12]:
count_vec = CountVectorizer(min_df=5, stop_words='english', max_features=1000)

In [13]:
train_vectors_desc = count_vec.fit_transform(data['SentimentText'])

In [14]:
train_vectors_desc

<49989x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 197892 stored elements in Compressed Sparse Row format>

In [15]:
data['SentimentText LENGTH'] = data['SentimentText'].apply(str).apply(len)

In [16]:
data['SentimentText COUNT'] = data['SentimentText'].apply(str).apply(lambda x: len(x.split(' ')))

In [17]:
data.head()

,Sentiment,SentimentText,SentimentText LENGTH,SentimentText COUNT
0,0.0,is so sad for my APL frie...,61,28
1,0.0,I missed the New Moon trail...,51,25
2,1.0,omg its already 7:30 :O,37,19
3,0.0,.. Omgaga. Im sooo im gunna CRy. I'...,132,36
4,0.0,i think mi bf is cheating on me!!! ...,53,24


In [18]:
data.drop(['SentimentText'],axis=1,inplace=True)

In [19]:
y = data['Sentiment']

In [20]:
data.drop(['Sentiment'],axis=1,inplace=True)

In [21]:
data.head()

,SentimentText LENGTH,SentimentText COUNT
0,61,28
1,51,25
2,37,19
3,132,36
4,53,24


In [22]:
x_train = hstack([data, train_vectors_desc])

In [23]:
model = xgb.XGBClassifier()

In [24]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5] ,'n_estimators': [100,500,700]}
ind_params = {'learning_rate': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 3, n_jobs = -1, verbose=3) 

In [25]:
optimized_GBM.fit(x_train, y)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  7.9min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.8),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5], 'n_estimators': [100, 500, 700]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=3)

In [26]:
optimized_GBM.grid_scores_

[mean: 0.65498, std: 0.00636, params: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100},
 mean: 0.70295, std: 0.00483, params: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 500},
 mean: 0.70854, std: 0.00470, params: {'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 700},
 mean: 0.65352, std: 0.00629, params: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100},
 mean: 0.70303, std: 0.00563, params: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 500},
 mean: 0.70988, std: 0.00471, params: {'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 700},
 mean: 0.65292, std: 0.00659, params: {'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 100},
 mean: 0.70287, std: 0.00541, params: {'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500},
 mean: 0.70964, std: 0.00522, params: {'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 700},
 mean: 0.66995, std: 0.00637, params: {'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100},


In [27]:
optimized_GBM.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=3, missing=None, n_estimators=700,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.8)

In [29]:
optimized_GBM.best_score_

0.7169577307007542

In [30]:
from xgboost import XGBClassifier
#Train the model
XGBOOST = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=3, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)


XGBOOST.fit(x_train,y)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=3, missing=None, n_estimators=500,
       n_jobs=1, nthread=-1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.8)

In [ ]:
#SAVING MODEL

In [31]:
import pickle

with open('XGBOOST.pkl', 'wb') as fid:
    pickle.dump(XGBOOST, fid,2)  